# Bad Photo Generator with more filters

### Generating bad photos from good photos

![image info](more_bad_images.png)

#### Choosing random images from a folder

In [1]:
# loading libraries
import cv2
import numpy as np
import os, random
from PIL import Image, ImageDraw

In [2]:
def choose_photos(filename, n_photos=-1):
    if isinstance(filename, set):
        photos = filename
    else:
        photos = set(os.listdir(folder))
    if n_photos == -1:
        return photos
    number_files = len(photos)
    if number_files < n_photos:
        return 'number of photos is larger than the photos in the folder/set'
    chosen_photos = set(random.sample(photos, n_photos))
    photos = photos - chosen_photos
    return photos, chosen_photos

In [3]:
folder = "flickr30k_images"
photos = choose_photos(folder)

In [4]:
len(photos)

31783

#### Creating "good" and "bad" folders

In [5]:
try:
    os.mkdir('good') 
except OSError as error: 
    print(error)

In [6]:
try:
    os.mkdir('bad')
except OSError as error: 
    print(error)

#### Copying the random chosen images into the "good" folder

In [7]:
n_good_photos = 224
photos, good_photos = choose_photos(photos, n_good_photos)

C:\Users\Eran\AppData\Local\Temp\ipykernel_18412\2906520519.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  chosen_photos = set(random.sample(photos, n_photos))


In [8]:
print('Chosen photos:', len(good_photos))
print('Remaining number of photos:', len(photos))
bool(photos & good_photos)

Chosen photos: 224
Remaining number of photos: 31559


False

In [9]:
import shutil

In [10]:
for photo in good_photos:
    shutil.copy(folder + '\\' + photo, 'good')

#### Filters for creating synthetic "bad" photos

In [11]:
# motion blur filter
#size - in pixels, size of motion blur
#angle - in degrees, direction of motion blur
def apply_motion_blur(image, size, angle):
    k = np.zeros((size, size), dtype=np.float32)
    k[ (size-1)// 2 , :] = np.ones(size, dtype=np.float32)
    k = cv2.warpAffine(k, cv2.getRotationMatrix2D( (size / 2 -0.5 , size / 2 -0.5 ) , angle, 1.0), (size, size) )
    k = k * (1.0 / np.sum(k))
    return cv2.filter2D(image, -1, k)

In [12]:
# random gaussian blur outer circle filter
def random_blur_out(image, radius=100):
    img_size = (image.shape[0], image.shape[1], 3)
    blurred_img = cv2.GaussianBlur(image, (21, 21), 0)
    mask = np.zeros(img_size, dtype=np.uint8)
    x, y = (random.randint(0, img_size[0]-radius), random.randint(0, img_size[1]-radius))
    mask = cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)
    return np.where(mask==np.array([255, 255, 255]), image, blurred_img)

In [13]:
# random gaussian blur inner circle filter
def random_blur_in(image, radius):
    img_size = (image.shape[0], image.shape[1], 3)
    blurred_img = cv2.GaussianBlur(image, (21, 21), 0)
    mask = np.zeros(img_size, dtype=np.uint8)
    x, y = (random.randint(0, img_size[0]-radius), random.randint(0, img_size[1]-radius))
    mask = cv2.circle(mask, (x, y), radius, (255, 255, 255), -1)
    return np.where(mask==np.array([0, 0, 0]), image, blurred_img)

In [14]:
# distance between two points (a and b)
def distance(a,b):
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

# return true/false if a point (c) is between two points (a and b)
def is_between(a,c,b):
    return distance(a,c) + distance(c,b) == distance(a,b)

# random gaussian semi-blur filter
def random_semi_blur(image):
    img_size = (image.shape[0], image.shape[1], 3)
    blurred_img = cv2.GaussianBlur(image, (21, 21), 0)
    mask = np.zeros(img_size, dtype=np.uint8)
    x_start, y_start = random.choice([(random.randint(0, img_size[0]), 0), (0, random.randint(0, img_size[1]))])
    x_end, y_end = (img_size[1] - x_start, img_size[0] - y_start)
    mask = cv2.line(mask, (x_start, y_start), (x_end, y_end), (255, 255, 255), 1)
    while True:
        seed = (random.randint(0, img_size[1]), random.randint(0, img_size[0]))
        if not is_between((x_start, y_start), seed, (x_end, y_end)):
            break
    rep_value = (255, 255, 255)
    mask = Image.fromarray(mask)
    ImageDraw.floodfill(mask, seed, rep_value, thresh=50)
    mask = np.asarray(mask)
    return np.where(mask==np.array([255, 255, 255]), image, blurred_img)

In [15]:
# illumination filter
def adjust_gamma(image, gamma=0.5):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)])
    return cv2.LUT(image.astype(np.uint8), table.astype(np.uint8))

In [16]:
# lens flare filter
def lens_flare(image, size=1, lens_flare_folder = '.\\lens flare\\'):
    
    # load random lens flare layer
    files = os.listdir(lens_flare_folder)
    lightsource = Image.open(lens_flare_folder + random.choice(files))
    
    # resize to input image size
    lightsource_size = (int(image.shape[1]*size), int(image.shape[0]*size))
    lightsource = lightsource.resize(lightsource_size)
    lightsource = Image.fromarray(np.uint8(lightsource))
    
    # Open Front Image
    frontImage = lightsource
    
    # Flip the light source layer Vertical & Horizontal randomly (or not)
    frontImage = random.choice([frontImage.transpose(Image.FLIP_LEFT_RIGHT), frontImage])
    frontImage = random.choice([frontImage.transpose(Image.FLIP_TOP_BOTTOM), frontImage])
    
    # Open Background Image
    background = Image.fromarray(np.uint8(image))

    # Convert image to RGBA
    frontImage = frontImage.convert("RGBA")

    # Convert image to RGBA
    background = background.convert("RGBA")

    # Calculate width to be at the center
    width = (background.width - frontImage.width) // 2

    # Calculate height to be at the center
    height = (background.height - frontImage.height) // 2

    # Paste the frontImage at (width, height)
    background.paste(frontImage, (width, height), frontImage)
    
    return np.asarray(background)

In [17]:
# random noise filter
def add_noise(image):
    mean = 0.0   # some constant
    std = 127.5    # some constant (standard deviation)
    noisy_img = image + np.random.normal(mean, std, (image.size[1], image.size[0], 3))
    noisy_img_clipped = np.clip(noisy_img, 0, 255)  # we might get out of bounds due to noise
    return noisy_img_clipped

In [18]:
def apply_filter(image, filter_name='motion_blur'):
    if filter_name == 'motion_blur':
        return apply_motion_blur(image, 10, 90)
    if filter_name == 'random_blur_out':
        return random_blur_out(image, radius=100)
    if filter_name == 'random_blur_in':
        return random_blur_in(image, radius=100)
    if filter_name == 'random_semi_blur':
        return random_semi_blur(image)
    if filter_name == 'bright':
        return adjust_gamma(image, 3)
    if filter_name == 'dark':
        return adjust_gamma(image, 0.3)
    if filter_name == 'flare':
        return lens_flare(image)
    if filter_name == 'noise':
        return add_noise(image)
    if filter_name == 'mix_1':
        return adjust_gamma(apply_motion_blur(lens_flare(image), 10, 90), 3)
    if filter_name == 'mix_2':
        return adjust_gamma(apply_motion_blur(lens_flare(image), 10, 90), 0.3)

#### Generating bad photos with filters

In [19]:
filters = ['motion_blur', 'bright', 'dark', 'flare', 'loss', 'mix_1', 'mix_2']
n_filtered_images = 1

for fil in filters:
    photos, chosen_photos_to_be_filtered = choose_photos(photos, n_filtered_images)
    for img_file in chosen_photos_to_be_filtered:
        img = Image.open(folder + '\\' + img_file).convert('RGB')
        img = np.array(img) 
        img = img[:, :, ::-1].copy()
        if fil != 'loss':
            cv2.imwrite('bad\\' + img_file[:-4] + '_' + fil + '.jpg', apply_filter(img , filter_name=fil))
        else:
            # low quality filter (jpeg loss)
            # The image quality, on a scale from 1 (worst) to 95 (best).
            # The default is 75. Values above 95 should be avoided;
            # 100 disables portions of the JPEG compression algorithm, and results in large files with hardly any gain in image quality.
            quality_val = 1
            img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
            Image.fromarray(img).save('bad\\' + img_file[:-4] + '_' + fil + '.jpg', 'JPEG', quality=quality_val)
    print('Generated ' + str(len(chosen_photos_to_be_filtered)) + ' for the ' + fil + ' filter')
print('Generated total of ' + str(n_filtered_images*len(filters)) + ' bad images')

Generated 1 for the motion_blur filter
Generated 1 for the bright filter
Generated 1 for the dark filter
Generated 1 for the flare filter
Generated 1 for the loss filter
Generated 1 for the mix_1 filter
Generated 1 for the mix_2 filter
Generated total of 7 bad images


C:\Users\Eran\AppData\Local\Temp\ipykernel_18412\2906520519.py:11: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  chosen_photos = set(random.sample(photos, n_photos))
